In [32]:
import requests
from bs4 import BeautifulSoup
import time
import threading
import urllib.request 
import webbrowser
import datetime
#File
from datetime import datetime
import os
from PIL import ImageTk
import PIL.Image
#UI
from tkinter import *
from tkinter import ttk
import tkinter.messagebox

In [33]:
class HTML():
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    def GetHTML(url):
        response = requests.get(url, headers=HTML.headers)
        # ip차단 방지를 위해 일시정지
        #time.sleep(0.9)
        return BeautifulSoup(response.text, "html.parser")
    
    def GetUnionTop(html):
        try:
            data_list = html.select('table.rank_table > tbody > tr > td')
            return data_list[2].text.replace(',', '')
        except(IndexError):
            return None

    def GetUnionBottom(html):
        try:
            data_list = html.select('table.rank_table > tbody > tr > td')
            return data_list[38].text.replace(',', '')
        except(IndexError):
            return None
        
    def GetCharacterInfoAll(html):
        # 모든 캐릭터 정보를 저장할 리스트 선언
        data_tuple_list = []

        # 모든 캐릭터 정보 추출
        character_info_list = html.select('table.rank_table > tbody > tr')

        for character_info in character_info_list:
            nick = character_info.select_one('a').text
            job = character_info.select_one('dd').text.replace(',', '')
            union = character_info.select('td')[2].text.replace(',', '')
            img_url = character_info.select_one('span.char_img > img')["src"]
            data_tuple_list.append((nick, job, union, img_url))

        return data_tuple_list

In [34]:
class Crawlling(HTML):
    is_running = False
    url = None
    search_min = 1
    search_max = 1
    
    def TempCode_InputInfo():
        GUI.server_combobox.set("크로아")
        GUI.union_min_entry.delete(0, len(GUI.union_min_entry.get()))
        GUI.union_max_entry.delete(0, len(GUI.union_max_entry.get()))
        GUI.union_min_entry.insert(0, '9055')
        GUI.union_max_entry.insert(0, '9055')
        
    def DeActivateReservation():
        Crawlling.is_running = False
        GUI.activate_button.config(text = "Start")
        GUI.SetProgressLabelMessage('', 100)
        GUI.EnableUi()
    
    def ValidateCrawlling_Thread():
        t1=threading.Thread(target=Crawlling.ValidateCrawlling)
        t1.setDaemon(True)
        t1.deamon = True
        t1.start()

    def ValidateCrawlling():
        # 테스트용 자동 입력
#         Crawlling.TempCode_InputInfo()

        if Crawlling.is_running == True:
            Crawlling.DeActivateReservation()
            tkinter.messagebox.showinfo("Message", "사용자 요청에 따라 작동을 중지합니다.")
            if tkinter.messagebox.askquestion ('Message','파일을 저장하시겠습니까?') == 'yes':
                CsvFile.SaveCsvFile()
            return
        else:
            GUI.activate_button.config(text = "Stop")
            Crawlling.is_running = True
        
        # 입력 오류 체크
        if Crawlling.IsExistInputError():
            Crawlling.DeActivateReservation()
            return
        
        # Ui 비활성화 및 CSV 초기화
        GUI.DisableUi()
        CsvFile.ClearCsvText()
        
        # 서버에 따른 URL 설정
        Crawlling.SetURL()
        
        # 탐색 범위 설정(1000단위, 100단위, 10단위로)
        Crawlling.SetSearchRange(1, 100000, 1000)
        Crawlling.SetSearchRange(Crawlling.search_min, Crawlling.search_max+100, 100)
        Crawlling.SetSearchRange(Crawlling.search_min, Crawlling.search_max+10, 10)
        if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
        GUI.SetProgressLabelMessage(f"검색 범위 설정 완료 [{Crawlling.search_min} ~ {Crawlling.search_max}]", 75)
        
        # 조건에 맞는 캐릭터 탐색
        Crawlling.FindSuspect()
        if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
        
        # 파일 저장 
        CsvFile.SaveCsvFile()
        if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
        
        # 결과 출력
        Crawlling.ShowResult()
        
        # 작업 종료
        Crawlling.DeActivateReservation()
        
    def IsExistInputError():
        if GUI.server_combobox.get() == '서버 선택':
            tkinter.messagebox.showerror("Message", "서버가 설정되지 않았습니다. \n서버를 설정해 주세요.")
            return True
        elif not GUI.union_min_entry.get() or not GUI.union_max_entry.get():
            tkinter.messagebox.showerror("Message", "유니온 범위가 입력되지 않았습니다. \n범위를 입력해 주세요.")
            return True
        else:
            try:
                int(GUI.union_min_entry.get())
                int(GUI.union_max_entry.get())
                return False
            except:
                tkinter.messagebox.showerror("Message", "유니온 범위 입력 중 숫자가 아닌 것이 입력되었습니다. \n숫자만 입력해 주세요.")
                return True

    def SetURL():
        server = GUI.GetServer()
        if server == '리부트2': server_code = 'w=1'
        elif server == '리부트1': server_code = 'w=2'
        elif server == '오로라': server_code = 'w=3'
        elif server == '레드': server_code = 'w=4'
        elif server == '이노시스': server_code = 'w=5'
        elif server == '유니온': server_code = 'w=6'
        elif server == '스카니아': server_code = 'w=7'
        elif server == '루나': server_code = 'w=8'
        elif server == '제니스': server_code = 'w=9'
        elif server == '크로아': server_code = 'w=10'
        elif server == '베라': server_code = 'w=11'
        elif server == '엘리시움': server_code = 'w=12'
        elif server == '아케인': server_code = 'w=13'
        elif server == '노바': server_code = 'w=14'
        Crawlling.url = f"https://maplestory.nexon.com/Ranking/Union?{server_code}&page="
        
    def SetSearchRange(start, end, increase):
        if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
        union_range_min = int(GUI.GetUnionMin())
        union_range_max = int(GUI.GetUnionMax())

        # 검색 범위 설정
        for i in range(start, end, increase):
            html = Crawlling.GetHTML(f"{Crawlling.url}{i}")
            union_top = int(Crawlling.GetUnionTop(html))
            union_bottom = int(Crawlling.GetUnionBottom(html))
            if union_top == None or union_bottom == None:
                Crawlling.search_max = i
                break
            if union_top > union_range_max:
                Crawlling.search_min = i
            if union_bottom < union_range_min:
                Crawlling.search_max = i
                break
            GUI.SetProgressLabelMessage(f"검색 범위를 설정 중입니다... [{i}P ~ {Crawlling.search_max+increase}P]", 60)
            if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
        print(f"현재 범위 : {Crawlling.search_min} ~ {Crawlling.search_max}")
        
    def FindSuspect():
        union_range_min = int(GUI.GetUnionMin())
        union_range_max = int(GUI.GetUnionMax())
        
        for i in range(Crawlling.search_min, Crawlling.search_max+1):
            html = Crawlling.GetHTML(f"{Crawlling.url}{i}")
            data_tuple_list = Crawlling.GetCharacterInfoAll(html)
            
            # 조건 일치 시 CsvFile 텍스트 내 저장 
            for data_tuple in data_tuple_list:
                character_union = int(data_tuple[2])
                if character_union >= union_range_min and character_union <= union_range_max:
                    nick = data_tuple[0]
                    job = data_tuple[1]
                    img_url = data_tuple[3]
                    CsvFile.AppendCsvText(f"{nick},{job},{character_union}{','*47}{img_url}")
                if Crawlling.is_running == False : Crawlling.DeActivateReservation(); return 0
                
                # character_union이 최소 조건보다 작으면 탐색 종료
                if character_union < union_range_min:
                    return
                
            GUI.SetProgressLabelMessage(f"조건에 맞는 캐릭터 탐색 중입니다... [{i}P / {Crawlling.search_max}P]", 50)
            
    
    def ShowResult():
        # 기존 소스 재활용
        character_info_list = CsvFile.csv_text.split('\n')
        
        def GoToMapleGG(event):
            nonlocal character_info_list
            nick = character_info_list[page].split(',')[0]
            url = 'https://maple.gg/u/'+nick
            webbrowser.open(url)
            
        def BeforeInfo():
            nonlocal page
            if page <= 1:
                page = page_max
                ShowCharacterInfo()
            else:
                page -= 1
                ShowCharacterInfo()

        def NextInfo():
            nonlocal page
            if page >= page_max:
                page = 1
                ShowCharacterInfo()
            else:
                page += 1
                ShowCharacterInfo()
                
        def ShowCharacterInfo():
            nonlocal character_info_list
            nick = character_info_list[page].split(',')[0]
            job = character_info_list[page].split(',')[1]
            union = character_info_list[page].split(',')[2]
            img_url = character_info_list[page].split(',')[49]
            
            #Get img raw
            character_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(img_url, stream=True).raw))
            
            #Info to Toplevel
            result_window.title("결과["+str(page)+'/'+str(page_max)+']')
            label_character_img.config(image = character_img)
            label_character_img.image = character_img # 파이썬 가비지 컬렉션의 이미지 삭제 방지        
            nick_label.config(text = nick)
            job_label.config(text = '직업 : '+job)
            union_label.config(text = '유니온 : '+union)
                
        #Toplevel Ui
        page = 1
        page_max = len(character_info_list)-2
        result_window = Toplevel()
        result_window.title("결과")
        result_window.geometry('230x345')
        result_window.option_add('*Font', '맑은고딕 9')
        result_window.resizable(False, False)
        label_character_img = Label(result_window)
        label_character_img.bind('<Button-1>', GoToMapleGG)
        label_img_caption = Label(result_window, text = '이미지 클릭 시 메이플GG로 이동합니다!', font = '맑은고딕 8')
        nick_label = Label(result_window)
        job_label = Label(result_window)
        union_label = Label(result_window)
        BeforeBtn = Button(result_window, width = 10, text = '이전', command = BeforeInfo)
        NextBtn = Button(result_window, width = 10, text = '다음', command = NextInfo)
        
        label_img_caption.pack()
        label_character_img.pack()
        nick_label.pack()
        union_label.pack()
        job_label.pack()
        BeforeBtn.place(x = 5, y = 317)
        NextBtn.place(x = 145, y = 317)
        ShowCharacterInfo()

In [35]:
class CsvFile():
    csv_text = "닉네임,직업,유니온\n"
    
    def ClearCsvText():
        CsvFile.csv_text = "닉네임,직업,유니온\n"
        
    def AppendCsvText(text):
        CsvFile.csv_text += f"{text}\n"
        
    def SaveCsvFile():
        GUI.SetProgressLabelMessage("결과 파일을 저장 중 입니다...", 85)
        file_name = f"[{GUI.GetServer()}] {GUI.GetUnionMin()} ~ {GUI.GetUnionMax()} 결과 ({datetime.today().strftime('%Y-%m-%d-%H%M%S')}).csv"
        with open(file_name,'w') as file:
            file.write(CsvFile.csv_text)
        tkinter.messagebox.showinfo("Result", f"{file_name}\n 파일이 저장되었습니다.")
        GUI.SetProgressLabelMessage("", 85)

In [37]:
class Update():
    present_version = '1.2.0'
    newest_version = None
    message = None
    update_url = None
    
    def CheckUpdate(self):
        try:
            url = 'https://blog.naver.com/PostView.naver?blogId=wkrnjs456&logNo=222945749414&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView&userTopListOpen=true&userTopListCount=5&userTopListManageOpen=false&userTopListCurrentPage=1'
            html = HTML.GetHTML(url)
            self.newest_version = html.select('div.se-component-content')[1].select('p')[1].text
            self.message = html.select('div.se-component-content')[1].select('p')[4].text
            self.update_url = html.select('div.se-component-content')[1].select('p')[7].text

            if self.present_version != self.newest_version:
                if tkinter.messagebox.askquestion ('Message', f"최신 버전({self.newest_version})이 발견되었습니다. \n(현재 버전 : {self.present_version})\n\n안내 사항 : {self.message} \n\n업데이트 하시겠습니까?") == 'yes':
                    self.GoDownloadURL()
        except:
            # 블로그 변화 시 오류 방지
            return
                
    def GoDownloadURL(self):
        webbrowser.open(self.update_url)

class UiFunction():
    def SetProgressLabelMessage(self, text, x):
        self.progress_label.config(text = text)
        self.progress_label.place(x = x, y = 230)
        
    def GetServer(self):
        return self.server_combobox.get()
        
    def GetUnionMin(self):
        return self.union_min_entry.get()
        
    def GetUnionMax(self):
        return self.union_max_entry.get()
    
    def EnableUi(self):
        self.server_combobox["state"] = "readonly"
        self.union_min_entry["state"] = "normal"
        self.union_max_entry["state"] = "normal"
        
    def DisableUi(self):
        self.server_combobox["state"] = "disabled"
        self.union_min_entry["state"] = "disabled"
        self.union_max_entry["state"] = "disabled"

class Ui(UiFunction, Update):
    def __init__(self):
        self.win = Tk()
        self.win.title(f'추노 - 본캐 추적 프로그램')
        self.win.geometry('330x300')
        self.win.option_add('*Font', '맑은고딕 10')
        self.win.resizable(False, False)
        
        # Default Ui
        Ui.MenuBar(self)
        Ui.MainImage(self)
        Ui.MainUi(self)
        
        # Check Update
        self.CheckUpdate()

    def MenuBar(self):
        def GoToMyBlog():
            webbrowser.open("https://blog.naver.com/PostList.naver?blogId=wkrnjs456&categoryNo=8&skinType=&skinId=&from=menu")
            
        def GoToMyGithub():
            webbrowser.open("https://github.com/JaGwonLim")
        self.menubar = Menu(self.win)
        self.menu_1 = Menu(self.menubar, tearoff=0)
        self.menu_1.add_command(label="Help", command = GoToMyBlog)
        self.menu_1.add_command(label="Github", command = GoToMyGithub)
        self.menubar.add_cascade(label="Option", menu=self.menu_1)
        self.win.config(menu=self.menubar)
        
    def MainImage(self):
        try:
            self.label_image = Label(self.win)
            main_image_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
            self.main_image = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_image_url, stream=True).raw).resize((350,70)))
            self.label_image = Label(self.win, image = self.main_image)
            self.label_image.pack()
        except:
            #사진 삭제 등으로 인한 오류 방지
            pass
    
    def MainUi(self):        
        # Server UI
        self.server_label = Label(self.win, text = '[서버 설정]')

        self.server_list =['루나', '스카니아', '엘리시움', '크로아', '베라', 
                      '오로라', '레드', '제니스', '이노시스', '유니온', 
                      '아케인', '노바', '리부트1', '리부트2'] 

        self.server_combobox = ttk.Combobox(self.win, 
                                       width = 11, 
                                       values = self.server_list, 
                                       state="readonly")
        self.server_combobox.set("서버 선택")
        
        self.server_label.place(x = 35 , y = 130)
        self.server_combobox.place(x = 140, y = 130) 
        
        # Union Range Ui
        self.union_label = Label(self.win, text = '[유니온 범위]')
        self.union_min_entry = Entry(self.win, width = 5)
        self.union_range_label = Label(self.win, text = '~')
        self.union_max_entry = Entry(self.win, width = 5)
        
        self.union_label.place(x = 30, y = 180)
        self.union_min_entry.place(x = 140, y = 180)
        self.union_range_label.place(x = 182, y = 180)
        self.union_max_entry.place(x = 200, y = 180)
        
        # Progress Ui
        self.progress_label = Label(self.win, text = '', font = "맑은고딕 8")
        self.progress_label.place(x = 40, y = 230)
        
        # Activate Button
        self.activate_button = Button(self.win, text = "Start", width = 25, height = 2, command = Crawlling.ValidateCrawlling_Thread)
        self.activate_button.place(x = 55, y = 250)
            
GUI = Ui()
GUI.win.mainloop()